<a href="https://colab.research.google.com/github/chancun2019/SVM-Machin-Learning/blob/main/Skripsi_2_Data_Cleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import pytz
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ast
import string
from wordcloud import WordCloud
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from googletrans import Translator
import io

In [14]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
cd 'drive/My Drive/Colab Notebooks/Output1'

/content/drive/My Drive/Colab Notebooks/Output1


In [17]:
data_mentah = pd.read_csv("data_crawling_terakhir_kali.csv")
selected_columns = ['tweet']
data_analisis = data_mentah[selected_columns]
data_analisis.head()

,tweet
0,Direktur jenderal badan kesehatan dunia who me...
1,Semua negara di dunia ini menyerukan pentingny...
2,@ato_uban @yo2thok Kirain tak ngefek vaksin pe...
3,@ato_uban @yo2thok Sebelumnya dikira demam ber...
4,@ato_uban @yo2thok Gejala timbul 5 hari setela...


### cleansing with regex

In [18]:
def clean_text(tweet):
    #menghapus RT tag
    t1 =re.sub('RT\s', '', tweet)
    #menghapus @_username
    #t2 =re.sub('\B@\w', '', t1)
    t2 =re.sub(r'\@([\w]+)',' ', t1)
    #mengganti emoji decode dengan spasi
    #t3 =emoji.demojize(t2)
    t3 = re.sub(r'\\u[a-zA-Z0-9]{4}', ' ', t2)
    #mengganti enter /n/ dengan spasi
    t4 =re.sub('\n\s', ' ', t3)
    t5 =re.sub('\n', ' ', t4)
    #Non-ascii
    t6 = re.sub(r'[^\x00-\x7F]+',' ', t5)
    #koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
    t7 = re.sub(r'([a-zA-Z])\1\1','\\1', t6)
    #replace URL
    t8 = re.sub(r'http[s]?\:\/\/.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', t7)
    t9 = re.sub(r'pic.twitter.com?.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', t8)
    # Tahap-10: konversi ke karakter huruf kecil
    t10 = t9.lower()
    #hapus tagar
    #t13 = re.sub(r'\#([\w]+)',' ', t12)
    t11 = re.sub(r'\#[a-zA-Z0-9_]+','', t10)
    #hapus angka
    t12 = re.sub(r'[0-9]+',' ', t11)
    # hapus simbol
    t13 = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', t12)
    # Tahap-8: spasi ganda (atau lebih) menjadi satu spasi
    t14 = re.sub(' +', ' ', t13)
    #Tahap-9: spasi di awal dan akhir kalimat
    t15 = re.sub(r'^[ ]|[ ]$','', t14)
    return t15

In [19]:
data_analisis = data_analisis['tweet'].apply(clean_text)
data_analisis.head(20)

0     direktur jenderal badan kesehatan dunia who me...
1     semua negara di dunia ini menyerukan pentingny...
2     kirain tak ngefek vaksin pertama baik saja sau...
3     sebelumnya dikira demam berdarah ada bintik di...
4        gejala timbul hari setelah vaksin gak langsung
5     ada rail clinic di stasiun delanggu klaten dar...
6                  soll info vaksin boster dong makasih
7     fess guyss info vaksin booster sekitar uns don...
8                                                      
9     hasil studi sebut faringitis jadi gejala utama...
10    tiap orang kan daya tahan tubuh imun lain itu ...
11    hmm yg dimasalahin disini bkn gk bs explore ho...
12    memang benar dalam lowongan pekerjaan saat ini...
13    lha siapa yang tahu kalau jadi kipi kalau tahu...
14    lha kalau tak vaksin tak bisa naik kereta api ...
15    ipar kk divaksin x kena covid juga masuk icu b...
16    di sini pcv aja kurang laku tun pdhal di kota ...
17    benar saya vaksin kali puasa dnah senen ka

In [20]:
#export DataFrame to CSV file
#data_analisis.to_csv('Dataset\data_tambahan.csv', index=False)
data_analisis.to_csv('data_tambahan-masih ada duplikasi data.csv', index=False)

In [21]:
# Membaca data yang sudah dibersihkan dari karakter yang tidak diperlukan
base = "data_tambahan-masih ada duplikasi data.csv"

### Stopword removal

In [22]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopwordList = factory.get_stop_words()
stopwords = factory.create_stop_word_remover()

In [23]:
data_analisis = data_analisis.apply(lambda x: " ".join(x for x in x.split()if x not in stopwordList))
data_analisis.head()

0    direktur jenderal badan kesehatan dunia who me...
1    semua negara dunia menyerukan pentingnya imuni...
2    kirain tak ngefek vaksin pertama baik saudara ...
3    sebelumnya dikira demam berdarah bintik paha s...
4               gejala timbul hari vaksin gak langsung
Name: tweet, dtype: object

In [24]:
# Menghapus data duplikat jika masih ada
data_analisis = data_analisis.drop_duplicates()

# Menghapus data kosong
data_analisis = data_analisis.dropna()

In [25]:
# Menyimpan teks preprosess clean
data_analisis.to_csv('data_preproses clean.csv', index=False)

In [26]:
data_analisis.head(20)

0     direktur jenderal badan kesehatan dunia who me...
1     semua negara dunia menyerukan pentingnya imuni...
2     kirain tak ngefek vaksin pertama baik saudara ...
3     sebelumnya dikira demam berdarah bintik paha s...
4                gejala timbul hari vaksin gak langsung
5     rail clinic stasiun delanggu klaten vaksin hin...
6                  soll info vaksin boster dong makasih
7     fess guyss info vaksin booster uns dong butuh ...
8                                                      
9     hasil studi sebut faringitis jadi gejala utama...
10    tiap orang kan daya tahan tubuh imun vaksin pe...
11    hmm yg dimasalahin disini bkn gk bs explore ho...
12    memang benar lowongan pekerjaan banyak mencant...
13    lha siapa tahu kalau jadi kipi kalau tahu saki...
14    lha kalau tak vaksin tak naik kereta api pesaw...
15    ipar kk divaksin x kena covid masuk icu bebera...
16    sini pcv aja kurang laku tun pdhal kota kabupa...
17    benar vaksin kali puasa dnah senen kamis i